In [1]:
import numpy as np
import pandas as pd
from itertools import permutations, product
from random import seed, shuffle, random
import math

## NUEVOS CONJUNTOS DE DATOS

Para crear los nuevos conjuntos de datos, uno para cada fase del modelo (train, validation y test), se seguirá la siguiente estrategia:

- Permutaciones sin repetición. En particular: 4 datasets para train, 3 para validation, 3 para test.
- Asignar un tipo a cada dataset (espontáneo, False; ataque, True)
- Duplicación de permutaciones invirtiendo el tipo
- Barajado de permutaciones

En cuanto al tamaño de los conjuntos de datos, se tiene, por un lado:

- 4 datasets train -> 24 (4!) x 2 (True/False) = 48 permutaciones

- 3 datasets validation -> 6 (3!) x 2 = 12 permutaciones

- 3 datasets test -> 3! = 6 (3!) x 2 = 12 permutaciones

Por otro lado, cada dataset se transforma en una serie temporal con 30 intervalos. Por lo tanto:

- Train: 48 permutaciones x 4 datasets/permutación x 30 intervalos/dataset = 5760 intervalos

- Validation: 12 permutaciones x 3 datasets/permutación x 30 intervalos/dataset = 1080 puntos

- Test: 12 permutaciones x 3 datasets/permutación x 30 intervalos/dataset = 1080 puntos

### PERMUTACIONES: orden datasets y tipos

In [2]:
#############################################
################### TRAIN ###################
#############################################

# Establecer la semilla
seed(10)

# Paso 1: Generar permutaciones de los datasets
nums_train = [0, 3, 5, 7]
perm_train = list(permutations(nums_train))
shuffle(perm_train)

# Paso 2: Generar permutaciones del tipo de dataset

# Generar una lista de booleanos con 50% True y 50% False (MISMO TAMAÑO QUE perm_train)
total_booleans = len(perm_train) * 4  
num_true = total_booleans // 2         
num_false = total_booleans - num_true  
# Crear lista
bool_flat_list = [True] * num_true + [False] * num_false
# Mezclar aleatoriamente
shuffle(bool_flat_list)
# Dividir en tuplas de 4 elementos
tipoDataset_train = [tuple(bool_flat_list[i:i+4]) for i in range(0, total_booleans, 4)]

# Paso 3: Duplicación de permutaciones invirtiendo el tipo (Si antes valía False, ahora vale True y viceversa)

# Lista complementaria de tipoDataset_train
tipoDataset_train_inv = [tuple(not x for x in comb) for comb in tipoDataset_train]
perm_train_extended = perm_train * 2                                    # Duplicamos las permutaciones
tipoDataset_train_extended = tipoDataset_train + tipoDataset_train_inv  # Unimos las listas complementarias

# Paso 4: Emparejar para no perder la correspondencia, barajamos y extraemos las listas
dataset_train = list(zip(perm_train_extended, tipoDataset_train_extended))
shuffle(dataset_train)
perm_train_extended, tipoDataset_train_extended = zip(*dataset_train)
perm_train_extended = list(perm_train_extended)
tipoDataset_train_extended = list(tipoDataset_train_extended)

In [3]:
dataset_train

[((3, 0, 7, 5), (True, False, True, False)),
 ((3, 5, 0, 7), (True, True, True, False)),
 ((3, 7, 0, 5), (True, False, False, True)),
 ((5, 7, 3, 0), (True, True, False, True)),
 ((5, 3, 7, 0), (True, False, True, False)),
 ((7, 5, 3, 0), (False, True, False, False)),
 ((5, 0, 3, 7), (False, False, False, True)),
 ((0, 7, 5, 3), (False, False, True, True)),
 ((0, 5, 3, 7), (True, False, False, False)),
 ((3, 5, 7, 0), (False, False, False, False)),
 ((3, 7, 5, 0), (True, True, False, False)),
 ((7, 0, 3, 5), (False, False, True, False)),
 ((5, 3, 0, 7), (False, False, False, True)),
 ((5, 0, 7, 3), (True, False, True, True)),
 ((7, 0, 5, 3), (False, True, True, False)),
 ((5, 3, 7, 0), (False, True, False, True)),
 ((5, 7, 0, 3), (True, True, True, True)),
 ((0, 5, 7, 3), (True, False, False, False)),
 ((7, 3, 5, 0), (True, False, False, True)),
 ((5, 7, 3, 0), (False, False, True, False)),
 ((7, 3, 5, 0), (False, True, True, False)),
 ((3, 5, 7, 0), (True, True, True, True)),
 ((7, 5,

In [4]:
#############################################
################ VALIDATION #################
#############################################

# Establecer la semilla
seed(68)

# Paso 1: Generar permutaciones de los datasets
nums_validation = [1, 6, 9]
perm_validation = list(permutations(nums_validation))
shuffle(perm_validation)

# Paso 2: Generar permutaciones del tipo de dataset

# Generar una lista de booleanos con 50% True y 50% False (MISMO TAMAÑO QUE perm_validation)
total_booleans = len(perm_validation) * 3  
num_true = total_booleans // 2         
num_false = total_booleans - num_true  
# Crear lista
bool_flat_list = [True] * num_true + [False] * num_false
# Mezclar aleatoriamente
shuffle(bool_flat_list)
# Dividir en tuplas de 3 elementos
tipoDataset_validation = [tuple(bool_flat_list[i:i+3]) for i in range(0, total_booleans, 3)]

# Paso 3: Duplicación de permutaciones invirtiendo el tipo (Si antes valía False, ahora vale True y viceversa)

# Lista complementaria de tipoDataset_validation
tipoDataset_validation_inv = [tuple(not x for x in comb) for comb in tipoDataset_validation]
perm_validation_extended = perm_validation * 2                                    # Duplicamos las permutaciones
tipoDataset_validation_extended = tipoDataset_validation + tipoDataset_validation_inv  # Unimos las listas complementarias

# Paso 4: Emparejar para no perder la correspondencia, barajamos y extraemos las listas
dataset_validation = list(zip(perm_validation_extended, tipoDataset_validation_extended))
shuffle(dataset_validation)
perm_validation_extended, tipoDataset_validation_extended = zip(*dataset_validation)
perm_validation_extended = list(perm_validation_extended)
tipoDataset_validation_extended = list(tipoDataset_validation_extended)

In [5]:
dataset_validation

[((6, 9, 1), (False, False, False)),
 ((9, 1, 6), (False, False, True)),
 ((6, 9, 1), (True, True, True)),
 ((1, 6, 9), (False, True, True)),
 ((1, 6, 9), (True, False, False)),
 ((1, 9, 6), (False, False, True)),
 ((6, 1, 9), (False, False, True)),
 ((6, 1, 9), (True, True, False)),
 ((9, 6, 1), (False, True, True)),
 ((1, 9, 6), (True, True, False)),
 ((9, 6, 1), (True, False, False)),
 ((9, 1, 6), (True, True, False))]

In [6]:
#############################################
################### TEST ####################
#############################################

# Establecer la semilla
seed(42)

# Paso 1: Generar permutaciones de los datasets
nums_test = [2, 4, 8]
perm_test = list(permutations(nums_test))
shuffle(perm_test)

# Paso 2: Generar permutaciones del tipo de dataset

# Generar una lista de booleanos con 50% True y 50% False (MISMO TAMAÑO QUE perm_test)
total_booleans = len(perm_test) * 3  
num_true = total_booleans // 2         
num_false = total_booleans - num_true  
# Crear lista
bool_flat_list = [True] * num_true + [False] * num_false
# Mezclar aleatoriamente
shuffle(bool_flat_list)
# Dividir en tuplas de 3 elementos
tipoDataset_test = [tuple(bool_flat_list[i:i+3]) for i in range(0, total_booleans, 3)]

# Paso 3: Duplicación de permutaciones invirtiendo el tipo (Si antes valía False, ahora vale True y viceversa)

# Lista complementaria de tipoDataset_test
tipoDataset_test_inv = [tuple(not x for x in comb) for comb in tipoDataset_test]
perm_test_extended = perm_test * 2                                    # Duplicamos las permutaciones
tipoDataset_test_extended = tipoDataset_test + tipoDataset_test_inv  # Unimos las listas complementarias

# Paso 4: Emparejar para no perder la correspondencia, barajamos y extraemos las listas
dataset_test = list(zip(perm_test_extended, tipoDataset_test_extended))
shuffle(dataset_test)
perm_test_extended, tipoDataset_test_extended = zip(*dataset_test)
perm_test_extended = list(perm_test_extended)
tipoDataset_test_extended = list(tipoDataset_test_extended)

In [7]:
dataset_test

[((8, 4, 2), (True, False, True)),
 ((2, 8, 4), (False, False, False)),
 ((8, 2, 4), (True, False, True)),
 ((4, 8, 2), (True, True, True)),
 ((4, 2, 8), (False, True, True)),
 ((2, 4, 8), (False, True, False)),
 ((2, 8, 4), (True, True, True)),
 ((2, 4, 8), (True, False, True)),
 ((4, 8, 2), (False, False, False)),
 ((8, 4, 2), (False, True, False)),
 ((8, 2, 4), (False, True, False)),
 ((4, 2, 8), (True, False, False))]

In [8]:
len(dataset_train)

48

In [9]:
len(dataset_validation)

12

In [10]:
len(dataset_test)

12

### FUNCIONES PARA LA CREACIÓN DE LOS DATOS

In [11]:
def add_dataset_ID_column(df, i):
    df['dataset_ID'] = i
    return df

def add_interval_column(df, long_interval):
    global EXECUTION
    bins = range(0, 3000 + long_interval, long_interval)  # 0 a 3000 ms en intervalos de long_interval ms
    # Agregamos una nueva columna al DataFrame con las etiquetas de los intervalos
    l = 3000 // long_interval
    df.loc[:, 'interval'] = EXECUTION * l + (pd.cut(df['timestamps'], bins=bins, labels=False) + 1)
    return df

def delete_unnecessary_columns(df):
    df_nuevo = df[["timestamps", "attack"]].copy()
    return df_nuevo

def add_spikes_column(df):
    df = df.groupby(['dataset_ID', 'interval', 'attack']).size().reset_index(name='spikes')
    return df

def add_class(df, tipo, long_interval):
    global EXECUTION
    df['class'] = 0
    if tipo:
        l = 3000 // long_interval
        intervalo = EXECUTION * l + math.ceil(625 / long_interval)
        df.loc[df['interval'] == intervalo, 'class'] = 1
    return df

def add_new_predictors(df):
    df.loc[:, 'differenceSpikes_1'] = df['spikes'].diff().fillna(0).astype(int)
    return df

def lectura_datasets(long_interval, lista, tipo):
    global EXECUTION

    # Rutas base del archivo CSV para ambos casos
    base_path_true = "./data/csv/output_FLO_quarter_flash_trial_{}_BKG_trial_9{}/spikes.csv"
    base_path_false = "./data/csv/output_flash_trial_{}_BKG_trial_9{}/spikes.csv"

    # DataFrame vacío para almacenar todos los datos
    combined_dataset = pd.DataFrame()

    for i, t in zip(lista, tipo):
        # Seleccionar la ruta base según el valor de la lista tipo
        base_path = base_path_true if t else base_path_false

        # Genera la ruta completa del archivo CSV para el valor actual de i
        file_path = base_path.format(i, i)

        try:
            # Lee el archivo CSV
            df = pd.read_csv(file_path, delimiter=";")
            df = delete_unnecessary_columns(df)
            df = add_dataset_ID_column(df, i)
            df = add_interval_column(df, long_interval)
            df = add_spikes_column(df)
            df = add_class(df, t, long_interval)
            # Concatena el dataframe al conjunto de datos consolidado
            combined_dataset = pd.concat([combined_dataset, df], ignore_index=True)
            EXECUTION = EXECUTION + 1

        except FileNotFoundError:
            print(f"Archivo no encontrado para i = {i}")

    return combined_dataset

In [12]:
# Variables globales para el conteo de los intervalos y su longitud
EXECUTION = 0
INTERVAL_LENGTH = 100

# Creación del conjunto de train
train_df = pd.DataFrame()
for perm, tipo in zip(perm_train_extended, tipoDataset_train_extended):
    train_df = pd.concat([train_df, lectura_datasets(INTERVAL_LENGTH, perm, tipo)], ignore_index=True)
train_df = add_new_predictors(train_df)
train_df.to_csv("./data/train.csv", index = False)

# Creación del conjunto de validation
validation_df = pd.DataFrame()
for perm, tipo in zip(perm_validation_extended, tipoDataset_validation_extended):
    validation_df = pd.concat([validation_df, lectura_datasets(INTERVAL_LENGTH, perm, tipo)], ignore_index=True)
validation_df = add_new_predictors(validation_df)
validation_df.to_csv("./data/validation.csv", index = False)

# Creación del conjunto de test
test_df = pd.DataFrame()
for perm, tipo in zip(perm_test_extended, tipoDataset_test_extended):
    test_df = pd.concat([test_df, lectura_datasets(INTERVAL_LENGTH, perm, tipo)], ignore_index=True)
test_df = add_new_predictors(test_df)
test_df.to_csv("./data/test.csv", index = False)